# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [69]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [70]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [71]:
# Your answer here
#178 rows, with 177 data-containing rows
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

_(ii)_ How many variables (columns) does the dataset contain?

In [72]:
# Your answer here
#14 columns as seen in the code above 

_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [73]:
# Your answer here
#integer (int64), and it has 3 unique levels/classes
wine_df["class"].unique()

array([0, 1, 2])


_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [74]:
# Your answer here
#12 predictor variables not including the ID for each entry

You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [75]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

the larger the scale of a predictor variable, the more weight it has in the final prediction. Therefore all variables must be brought into a standard scale so no sinlge predictor variables has an inflated effect on the prediction outcome. 

(ii) Why did we elect not to standard our response variable `Class`?

Because class is a 'classification' type variable, not a numerical variable. It has a fixed value and the outcome can only be one of three. It is easy to interpret and standarizing the outcome is meaningless and can make interpretation difficult. 

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

A random seed number is used to introduce randomness into the model to make our evaluation fairer. 

The random number generator needs a 'seed value' to start with. The number we choose itself is not particularly important, but the number must remain the same through out our model deployment as we want the random numbers generated to remain the same. Changing the 'seed number' changes the random numbers outputted. 

In [76]:
np.random.seed(5)

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [77]:
# Do not touch
np.random.seed(123)
# Create a random vector of True and False values to split the data
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])

# Your code here...

print(split)
wine_df_train = predictors_standardized[split]
wine_df_test = predictors_standardized [~split]
#wine_df_train.info()
#wine_df_test.info()

outcomes = wine_df.iloc[:,-1: ]
outcome_test = outcomes[~split]
outcome_train = outcomes[split]
#outcome_test.info()
#outcome_train.info()

[ True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False  True  True  True  True  True False  True  True False
  True  True  True False  True  True  True  True  True  True False  True
  True  True  True  True False  True False  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
 False False  True  True  True  True  True False  True False  True  True
  True  True  True  True  True  True  True  True  True  True False  True
 False  True  True  True  True False  True  True  True  True  True False
 False  True  True  True False False  True  True False  True  True  True
  True False False  True  True  True  True  True  True  True False  True
  True False  True  True  True False  True False  True  True  True  True
  True False  True  True  True False  True  True  T

code from: https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas

#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [78]:

list(outcome_train["class"])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [79]:
# Your code here...

knn = KNeighborsClassifier(n_neighbors=5)
X = wine_df_train
y = list(outcome_train["class"])

parms_grid = {
    "n_neighbors": range(1, 50, 2),
}

wine_tune_grid =GridSearchCV(
    estimator= knn,
    param_grid= parms_grid,
    cv = 10

)
wine_tune_grid.fit(
    wine_df_train,
    list(outcome_train["class"])
)

accuracy_grid = pd.DataFrame(wine_tune_grid.cv_results_)
accuracy_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000752,0.000468,0.001027,0.000236,1,{'n_neighbors': 1},0.857143,0.928571,0.928571,0.928571,1.0,0.928571,1.0,1.000000,0.857143,0.928571,0.935714,0.050000,20
1,0.000519,0.000091,0.000928,0.000099,3,{'n_neighbors': 3},0.928571,1.000000,0.928571,0.928571,1.0,1.000000,1.0,0.928571,0.857143,0.928571,0.950000,0.045737,10
2,0.000508,0.000062,0.000905,0.000061,5,{'n_neighbors': 5},0.928571,0.928571,0.928571,1.000000,1.0,0.928571,1.0,1.000000,0.928571,1.000000,0.964286,0.035714,2
3,0.000500,0.000054,0.000904,0.000069,7,{'n_neighbors': 7},1.000000,0.928571,0.928571,1.000000,1.0,0.928571,1.0,1.000000,0.928571,1.000000,0.971429,0.034993,1
4,0.000494,0.000067,0.000953,0.000135,9,{'n_neighbors': 9},1.000000,0.928571,0.928571,0.928571,1.0,0.928571,1.0,1.000000,0.928571,1.000000,0.964286,0.035714,2
5,0.000498,0.000027,0.000927,0.000112,11,{'n_neighbors': 11},1.000000,0.857143,0.928571,0.928571,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.964286,0.047916,2
6,0.000486,0.000024,0.000896,0.000034,13,{'n_neighbors': 13},1.000000,0.857143,0.928571,0.928571,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.964286,0.047916,2
7,0.000505,0.000070,0.000945,0.000104,15,{'n_neighbors': 15},0.928571,0.857143,0.928571,0.928571,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.957143,0.047380,8
8,0.000494,0.000040,0.000910,0.000049,17,{'n_neighbors': 17},0.928571,0.857143,0.928571,0.928571,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.957143,0.047380,8
9,0.000497,0.000050,0.000916,0.000044,19,{'n_neighbors': 19},1.000000,0.928571,0.857143,0.928571,1.0,0.928571,1.0,1.000000,1.000000,1.000000,0.964286,0.047916,2


In [80]:
wine_tune_grid.best_params_

{'n_neighbors': 7}

#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [81]:
# Your code here...
#git the model on the training data
knn = KNeighborsClassifier(n_neighbors=7)
X = wine_df_train
y = list(outcome_train["class"])
knn.fit(X, y)

returned_dictionary = cross_validate(
    estimator=knn,
    cv=10,   
    X=X,
    y=y
)

cv_10_wine_train = pd.DataFrame(returned_dictionary)    

cv_10_wine_train


,fit_time,score_time,test_score
0,0.004057,0.003585,1.000000
1,0.001720,0.002625,0.928571
2,0.000956,0.001138,0.928571
3,0.000526,0.000906,1.000000
4,0.000490,0.000875,1.000000
5,0.000479,0.000881,0.928571
6,0.000476,0.001161,1.000000
7,0.000953,0.001322,1.000000
8,0.000700,0.000985,0.928571
9,0.000494,0.001235,1.000000


In [83]:
#test the model on the testing data

wine_df_test["predicted"] = knn.predict(wine_df_test)
wine_df_test

/var/folders/2r/1l6yc2bx3m56rxmdpbz1z5w80000gn/T/ipykernel_4064/4040661365.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_df_test["predicted"] = knn.predict(wine_df_test)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,predicted
6,1.716255,-0.418624,0.305159,-1.469878,-0.262708,0.328298,0.492677,-0.498407,0.681738,0.083015,0.274431,1.367689,1.729520,0
21,-0.087232,1.313866,1.036228,-0.268738,0.158572,0.184088,0.382241,-0.901297,0.681738,-0.241413,0.318304,1.282942,0.073583,0
37,0.061000,-0.616110,0.670693,-0.448909,-0.122282,0.248181,0.402320,-0.578985,-0.264388,-0.349555,0.713164,-0.143625,1.140389,0
38,0.085705,-0.750759,-0.974210,-1.199622,-0.122282,0.168065,0.613153,-0.659563,-0.387033,-0.587469,0.976405,0.110615,0.869707,0
44,0.061000,-0.508390,-0.974210,-0.749194,0.509638,1.129464,0.974581,-0.659563,0.769342,-0.007825,-0.339797,1.042827,0.439800,0
47,1.110975,-0.589180,-0.901103,-1.049479,0.088358,1.289697,1.366128,-1.223610,0.962071,0.450699,-0.208177,1.014578,0.758249,0
51,1.024507,-0.616110,0.853460,-0.689137,-0.403135,0.248181,0.964541,-1.143031,1.224884,0.234414,1.239645,1.071076,1.649908,0
58,0.888627,-0.813595,0.487926,-0.839280,0.579852,1.770396,1.647239,-1.384766,0.786863,0.753498,-0.295924,0.364855,1.713598,0
64,-1.026033,-0.795642,0.597587,-0.148624,0.298998,-0.649124,-0.280377,0.710264,-0.982742,-0.911896,2.160986,-0.539109,-1.247982,1
66,0.135116,-1.190614,-2.436346,-1.349764,-1.526548,1.097417,1.155295,-0.820719,1.207363,0.104643,0.713164,0.802712,-0.779861,1


In [84]:
#evalute the accuracy of the model
accuracy = accuracy_score(outcome_test, wine_df_test["predicted"] )
print( f"accuracy_score function - accuracy {accuracy}")

accuracy_score function - accuracy 0.9736842105263158


code from https://www.geeksforgeeks.org/difference-between-score-and-accuracy_score-methods-in-scikit-learn/

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applying_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-4-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
